In [2]:
import pandas as pd

import os
import sys

sys.path.append(os.path.abspath("../../src"))
from retrieval import retrieve_text

In [ ]:
drug_library_synonyms_path = "../../data/drug_library/harmonized_synonyms/drug_library_synonyms_harmonized.parquet"
drug_library_data = pd.read_parquet(drug_library_synonyms_path)

# If an NCBI API key is provided, the runtime will be ~8x faster
NCBI_API_KEY = "bdd2f83e20dc27d1e257d3896d036fd0a108"
drug_library_data_retrieved= retrieve_text(drug_library_data, ncbi_key=NCBI_API_KEY)

drug_library_data_retrieved.to_parquet("../../data/drug_library/retrieved/first_try/drug_library_retrieved.parquet")

In [ ]:
retrieved_information = pd.read_parquet("../../data/drug_library/retrieved/first_try/drug_library_retrieved.parquet")
retrieved_information_isna = retrieved_information[retrieved_information["TEXT"].isna()]

retrieved_information_retry = retrieve_text(retrieved_information_isna, ncbi_key=NCBI_API_KEY, max_cores=4)
retrieved_information_retry.to_parquet("../../data/drug_library/retrieved/second_try/drug_library_retrieved.parquet")

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


/opt/homebrew/anaconda3/envs/chemsource-data-analysis/lib/python3.13/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /opt/homebrew/anaconda3/envs/chemsource-data-analysis/lib/python3.13/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
/opt/homebrew/anaconda3/envs/chemsource-data-analysis/lib/python3.13/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem,

In [ ]:
retrieved_information = pd.read_parquet("../../data/drug_library/retrieved/second_try/drug_library_retrieved.parquet")
retrieved_information_isna = retrieved_information[retrieved_information["TEXT"].isna()]

retrieved_information_retry = retrieve_text(retrieved_information_isna, ncbi_key=None, max_cores=4)
retrieved_information_retry.to_parquet("../../data/drug_library/retrieved/third_try/drug_library_retrieved.parquet")

/opt/homebrew/anaconda3/envs/chemsource-data-analysis/lib/python3.13/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /opt/homebrew/anaconda3/envs/chemsource-data-analysis/lib/python3.13/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
/opt/homebrew/anaconda3/envs/chemsource-data-analysis/lib/python3.13/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem,

In [13]:
first_retrieval = "../../data/drug_library/retrieved/first_try/drug_library_retrieved.parquet"
second_retrieval = "../../data/drug_library/retrieved/second_try/drug_library_retrieved.parquet"
third_retrieval = "../../data/drug_library/retrieved/third_try/drug_library_retrieved.parquet"

merged_data = pd.concat([
    pd.read_parquet(first_retrieval),
    pd.read_parquet(second_retrieval),
    pd.read_parquet(third_retrieval)
])

merged_data = merged_data.drop_duplicates(subset=["FEATURE_ID"], keep="last")
merged_data["TEXT"] = merged_data["TEXT"].apply(lambda x: x if str(x).strip() != "" else None)
merged_data.to_parquet("../../data/drug_library/retrieved/_merged/drug_library_all_retrieved.parquet")